In [ ]:
# Adicione isso junto com os outros imports
import xgboost as xgb
from xgboost import XGBClassifier

import numpy as np
import pandas as pd
import os

import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.datasets import fetch_openml

from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict

from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.metrics import ConfusionMatrixDisplay

from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.base import clone

In [2]:
BASE_PATH = "/kaggle/input/the-oxford-iii-t-pet-dataset/" 
IMAGES_PATH = os.path.join(BASE_PATH, "images/images")
ANNOTATIONS_PATH = os.path.join(BASE_PATH, "annotations/annotations")

IMG_SIZE = (128, 128)

In [3]:
import os
import cv2
import numpy as np
from sklearn.utils import shuffle

def load_data_complete(folder_path, img_size=(128, 128)):
    X = []
    y_especie = [] # 0: Gato, 1: Cachorro
    y_raca = []
    
    print("Carregando imagens e extraindo raças...")
    
    for filename in os.listdir(folder_path):
        if filename.endswith((".jpg", ".png", ".jpeg")):
            img_path = os.path.join(folder_path, filename)
            try:
                img = cv2.imread(img_path)
                if img is None: continue
                
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                img = cv2.resize(img, img_size)
                
                if filename[0].islower():
                    label = 1 
                else:
                    label = 0 
                
                breed = filename.rsplit('_', 1)[0]
                
                X.append(img.flatten())
                y_especie.append(label)
                y_raca.append(breed)
                
            except Exception as e:
                pass

    return np.array(X), np.array(y_especie), np.array(y_raca)

In [4]:
X, y, y_breed = load_data_complete(IMAGES_PATH, IMG_SIZE)

Carregando imagens e extraindo raças...


Corrupt JPEG data: 240 extraneous bytes before marker 0xd9
Corrupt JPEG data: premature end of data segment


In [5]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import numpy as np


print(f"Total de imagens: {len(X)}")
print(f"Exemplos de raças no dataset: {np.unique(y_breed)[:5]}")

le = LabelEncoder()
y_encoded = le.fit_transform(y_breed)

class_names = le.classes_
print(f"Total de Classes (Raças): {len(class_names)}")

Total de imagens: 7390
Exemplos de raças no dataset: ['Abyssinian' 'Bengal' 'Birman' 'Bombay' 'British_Shorthair']
Total de Classes (Raças): 37


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
)

print(f"Treino: {X_train.shape}, Teste: {X_test.shape}")

Treino: (5912, 49152), Teste: (1478, 49152)


In [ ]:
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, ConfusionMatrixDisplay
from skimage.feature import local_binary_pattern, hog
import matplotlib.pyplot as plt

# --- 4. FUNÇÕES DE EXTRAÇÃO (HOG) ---

def extract_hog_features(X, img_size):
    features_list = []
    print(f"   > Extraindo HOG de {len(X)} imagens...")
    for i in range(len(X)):
        img = X[i].reshape(img_size[0], img_size[1], 3)
        gray = cv2.cvtColor(img.astype('uint8'), cv2.COLOR_BGR2GRAY)
        f = hog(gray, orientations=9, pixels_per_cell=(8, 8),
                cells_per_block=(2, 2), block_norm='L2-Hys', visualize=False)
        features_list.append(f)
    return np.array(features_list)


In [8]:
print("1. Extraindo Features do TREINO (HOG)...")
X_train_hog = extract_hog_features(X_train, IMG_SIZE)


print("2. Extraindo Features do TESTE (HOG)...")
X_test_hog = extract_hog_features(X_test, IMG_SIZE)

print(f"Shape Final das Features: {X_train_hog.shape}")

1. Extraindo Features do TREINO (HOG)...
   > Extraindo HOG de 5912 imagens...
2. Extraindo Features do TESTE (HOG)...
   > Extraindo HOG de 1478 imagens...
Shape Final das Features: (5912, 8100)


In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.manifold import LocallyLinearEmbedding
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import time

print(f"Dimensão ANTES do LLE: {X_train_hog.shape[1]} features")

print("\nNormalizando dados (StandardScaler)...")
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_hog)
X_test_scaled = scaler.transform(X_test_hog)

Dimensão ANTES do LLE: 8100 features

7. Normalizando dados (StandardScaler)...


In [ ]:
print("Aplicando LLE (Isso pode demorar)...")

lle = LocallyLinearEmbedding(n_components=50, n_neighbors=15, random_state=42, n_jobs=-1)

start = time.time()
X_train_lle = lle.fit_transform(X_train_scaled)
print(f"   LLE Treino concluído em {time.time() - start:.1f}s")

print("   Aplicando LLE no Teste...")
X_test_lle = lle.transform(X_test_scaled)

print(f"Dimensão DEPOIS do LLE: {X_train_lle.shape[1]} features")

8. Aplicando LLE (Isso pode demorar)...
   LLE Treino concluído em 30.6s
   Aplicando LLE no Teste...
Dimensão DEPOIS do LLE: 50 features


In [ ]:
from sklearn.ensemble import RandomForestClassifier

print("Treinando Random Forest para 37 Raças...")

rf_all_breeds = RandomForestClassifier(
    n_estimators=500, 
    random_state=42, 
    n_jobs=-1
)

rf_all_breeds.fit(X_train_lle, y_train)
print("Modelo treinado com sucesso!")

3. Treinando Random Forest para 37 Raças...
Modelo treinado com sucesso!


In [ ]:
from sklearn.metrics import classification_report, ConfusionMatrixDisplay, accuracy_score
import matplotlib.pyplot as plt

y_pred_all = rf_all_breeds.predict(X_test_lle)

acc = accuracy_score(y_test, y_pred_all)
print(f"=== RESULTADO GERAL (37 RAÇAS) ===")
print(f"Acurácia Global: {acc:.2%}")
print("(Note que o chute aleatório seria apenas 2.7%)")

print("\nRelatório por Raça:")
print(classification_report(y_test, y_pred_all, target_names=class_names))

=== RESULTADO GERAL (37 RAÇAS) ===
Acurácia Global: 6.02%
(Note que o chute aleatório seria apenas 2.7%)

Relatório por Raça:
                            precision    recall  f1-score   support

                Abyssinian       0.19      0.15      0.17        40
                    Bengal       0.06      0.07      0.06        40
                    Birman       0.05      0.10      0.07        40
                    Bombay       0.24      0.25      0.25        40
         British_Shorthair       0.05      0.05      0.05        40
              Egyptian_Mau       0.10      0.12      0.11        40
                Maine_Coon       0.03      0.03      0.03        40
                   Persian       0.05      0.07      0.06        40
                   Ragdoll       0.08      0.05      0.06        40
              Russian_Blue       0.02      0.03      0.02        40
                   Siamese       0.06      0.05      0.05        40
                    Sphynx       0.07      0.10      0.08

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import ConfusionMatrixDisplay
from IPython.display import FileLink, display

# ==============================================================================
# GRÁFICO 1: RECALL (Normalizado por Linha)
# ==============================================================================
fig, ax = plt.subplots(figsize=(20, 20))

ConfusionMatrixDisplay.from_predictions(
    y_test, 
    y_pred_all, 
    normalize="true",       
    display_labels=class_names, 
    cmap="viridis", 
    xticks_rotation='vertical',
    values_format=".1f",    
    ax=ax,
    colorbar=False
)

ax.set_title("Matriz de Confusão - Recall (Sensibilidade)", fontsize=20)
plt.tight_layout()

file_recall = "matriz_recall.pdf"

plt.savefig(file_recall, format='pdf', bbox_inches='tight')
plt.close()

print(f"Arquivo gerado: {file_recall}")
# GERA O LINK PARA DOWNLOAD
display(FileLink(file_recall))

# ==============================================================================
# GRÁFICO 2: PRECISÃO (Normalizado por Coluna)
# ==============================================================================
fig, ax = plt.subplots(figsize=(20, 20))

ConfusionMatrixDisplay.from_predictions(
    y_test, 
    y_pred_all, 
    normalize="pred",       
    display_labels=class_names, 
    cmap="magma",           
    xticks_rotation='vertical',
    values_format=".1f",
    ax=ax,
    colorbar=False
)

ax.set_title("Matriz de Confusão - Precisão (Confiança)", fontsize=20)
plt.tight_layout()

file_precisao = "matriz_precisao.pdf"

plt.savefig(file_precisao, format='pdf', bbox_inches='tight')
plt.close()

print(f"Arquivo gerado: {file_precisao}")
# GERA O LINK PARA DOWNLOAD
display(FileLink(file_precisao))

Arquivo gerado: matriz_recall.pdf


/kaggle/working/matriz_recall.pdf

Arquivo gerado: matriz_precisao.pdf


/kaggle/working/matriz_precisao.pdf